In [ ]:
import pandas as pd
import requests
import urllib3
import time
from multiprocessing import Pool

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_urls(csv_file):
  urls=[]
  csv_test = pd.read_csv(csv_file,encoding = 'CP949')
  for url in csv_test["urls.website"]:
    url = url.split(',')
    url[0] = url[0].replace("[", "")
    url[0] = url[0].replace("]", "")
    url = url[0].replace("'", "")
    urls.append(url)
  return urls


def valid_url(data):
  headers={
      "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
      "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
      "Accept-Encoding" : "gzip, deflate, br",
      "Cache-Control" : "max-age=0",
      "Connection" : "keep-alive",
      "Referer" : "https://colab.research.google.com/",
      "Upgrade-Insecure-Requests" : "1",
      "sec-ch-ua" : "\"Google Chrome\";v=\"93\", \" Not;A Brand\";v=\"99\", \"Chromium\";v=\"93\"",
      "sec-ch-ua-mobile" : "?0",
      "sec-ch-ua-platform" : "\"Windows\""
      }
      
  url = data['urls.website']
  url = url.split(',')
  url[0] = url[0].replace("[", "")
  url[0] = url[0].replace("]", "")
  url = url[0].replace("'", "")

  
  if int(data['cmc_rank']) %50  == 0:
    print(data['cmc_rank'])

  
  try:
    response = requests.get(url,headers=headers,verify=False)
#    print("status code = %d, url = %s"%(response.status_code,url))
    if response.status_code not in [200,403,503]:
       print("idx : %s, status code = %d, url = %s"%(data['cmc_rank'],response.status_code,url))
       return 0
    else:
      return 1
  except Exception as e:
#      print(e)
      print("idx : %s, Excepterror url = %s"%(data['cmc_rank'],url))
      return 0



  
if __name__=='__main__':
    csv_test = pd.read_csv('3301.csv',encoding = 'CP949')
    data = csv_test.to_dict('records')
    start_time = time.time()
    pool = Pool() # 4개의 프로세스를 사용합니다.
    result = pool.map(valid_url,data)
    for i in range(len(data)):
      data[i]['valid_url'] = result[i]
    list_df = pd.DataFrame(data)
    list_df.to_csv('result2.csv',index=False)
    print("--- %s seconds ---" % (time.time() - start_time))